In [ ]:
# Add fairhousingguardrail package to system path

import os
import sys

dir_path = os.path.dirname(os.path.realpath("./")) + "/fair_housing_guardrail"

if dir_path not in sys.path:
    sys.path.insert(1, dir_path)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from data.json_dataset import JsonDataset
from utils.helper import load_config, load_dataset, load_tokenizer
from utils.fair_housing_classification import FairHousingGuardrailClassification

In [ ]:
CONFIG_FILE_PATH = "configs/train-config.yaml"
config = load_config(CONFIG_FILE_PATH)
tokenizer = load_tokenizer()
train_dataset, test_dataset = load_dataset(config, tokenizer)

In [ ]:
fh_classifier = FairHousingGuardrailClassification(config, tokenizer, train_dataset, test_dataset)
train_loss, eval_loss = fh_classifier.train()

print("Train Loss")
print(train_loss)

print("Validation Loss")
print(eval_loss)

# Plot loss
_train_loss, steps = zip(*train_loss)
_eval_loss, steps = zip(*eval_loss)

fig = go.Figure()
fig.add_trace(go.Scatter(x=steps, y=_train_loss, mode="lines+markers", name="Training Loss"))
fig.add_trace(go.Scatter(x=steps, y=_eval_loss, mode="lines+markers", name="Validation Loss"))

fig.update_layout(title="Loss", xaxis_title="Training Steps", yaxis_title="Loss")
fig.show()

# Save model and tokenizer
fh_classifier.trainer.save_model()

fh_classifier.model.save_pretrained(config["output_data"]["model_output_dir"])
fh_classifier.tokenizer.save_pretrained(config["output_data"]["tokenizer_output_dir"])